In [1]:
%cd ../

/home/hoanghu/projects/Food-Waste-Optimization/experiments_hoangle


In [2]:
import os
from pathlib import Path

import pandas as pd
import psycopg as pg
from loguru import logger
from psycopg import sql
from dotenv import load_dotenv

from utils import Paths

In [3]:
load_dotenv()

True

# Load tables

In [4]:
dishes = pd.read_excel(Paths.dim_dishes())

dishes.head()

,meal_id,restaurant,category,dish
0,1,Chemicum,chicken,BBQ-Broilerikastiketta
1,2,Chemicum,chicken,Broileria appelsiini-currykastikkeessa
2,3,Chemicum,chicken,Broileria pekonikastikkeessa
3,4,Chemicum,chicken,Broileria pestokastikkeessa
4,5,Chemicum,chicken,Broilerinkoipea


In [5]:
dish_biowaste = pd.read_excel(Paths.pred_biowaste())

dish_biowaste.head()

,meal_id,waste
0,1,0.043050
1,2,0.065768
2,3,0.010000
3,4,0.010000
4,5,0.235866


In [6]:
pred_n_pcs_per_dish = pd.read_excel(Paths.pred_n_pcs_per_dish())
pred_n_pcs_per_dish.head()

,meal_id,date,pcs
0,1,2024-09-02,202
1,2,2024-09-02,76
2,3,2024-09-02,129
3,4,2024-09-02,78
4,5,2024-09-02,94


In [7]:
pred_n_pcs_whole = pd.read_excel(Paths.pred_n_pcs_whole())
pred_n_pcs_whole.head()

,date,pcs,restaurant
0,2024-09-02,728.91,Chemicum
1,2024-09-03,729.82,Chemicum
2,2024-09-04,707.55,Chemicum
3,2024-09-05,660.46,Chemicum
4,2024-09-06,678.02,Chemicum


In [8]:
co2 = pd.read_excel(Paths.pred_co2())
co2.head()

,meal_id,co2
0,1,0.57
1,2,0.64
2,3,0.99
3,4,0.54
4,5,1.28


In [12]:
paths = Path("recommendation/menus_recommended").glob("*.xlsx")

menus   = pd.concat([pd.read_excel(f) for f in paths], ignore_index=True)

menus.head()

,dish_1,dish_2,dish_3,dish_4,total_co2,total_waste,total_pcs_from_dishes,co2_per_customer,waste_per_customer,fitness,date,restaurant
0,246,286,318,321,62.04,1.40,140,0.443143,0.01,1.136857,2025-01-01,Exactum
1,312,313,178,321,61.92,1.41,141,0.439149,0.01,1.143159,2025-01-01,Exactum
2,291,309,316,321,62.69,1.40,140,0.447786,0.01,1.146143,2025-01-01,Exactum
3,246,249,178,316,58.52,1.36,136,0.430294,0.01,1.167176,2025-01-01,Exactum
4,246,291,178,321,60.17,1.47,147,0.409320,0.01,1.169240,2025-01-01,Exactum


# SQL

## Load env

In [10]:
USER = os.getenv('DB_USER')
PWD = os.getenv('DB_PWD')
PORT = os.getenv('DB_PORT')
HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')

### Fetch

In [14]:
table_name = "pieces_whole"

try:
    with pg.connect(
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT,
        dbname=DB_NAME,
    ) as conn:
        with conn.cursor() as cur:
            stmt = (
                sql
                .SQL("""
                    select
                        *
                    from
                        {table}
                    limit 10
                    ;""")
                .format(
                    table=sql.Identifier(table_name),
                    # res_type=sql.Literal(res_type),
                )
            )
            cur.execute(stmt)

            ret = cur.fetchall()

except pg.OperationalError as e:
    logger.error(f"Connect to DB got error: {e}")\
    
ret

[(1, datetime.date(2024, 9, 2), 'Chemicum', 728.91),
 (2, datetime.date(2024, 9, 3), 'Chemicum', 729.82),
 (3, datetime.date(2024, 9, 4), 'Chemicum', 707.55),
 (4, datetime.date(2024, 9, 5), 'Chemicum', 660.46),
 (5, datetime.date(2024, 9, 6), 'Chemicum', 678.02),
 (6, datetime.date(2024, 9, 9), 'Chemicum', 704.77),
 (7, datetime.date(2024, 9, 10), 'Chemicum', 697.67),
 (8, datetime.date(2024, 9, 11), 'Chemicum', 675.29),
 (9, datetime.date(2024, 9, 12), 'Chemicum', 650.1),
 (10, datetime.date(2024, 9, 13), 'Chemicum', 699.27)]

## Insert tables

### table 'dishes'

In [17]:
table_name = "dishes"
values = [(r.meal_id, r.restaurant, r.category, r.dish) for r in dishes.itertuples()]

try:
    with pg.connect(
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT,
        dbname=DB_NAME,
    ) as conn:
        with conn.cursor() as cur:
            stmt = (
                sql.SQL(
                    """
                    insert into {table} 
                    (meal_id, restaurant, category, dish)
                    values ({placeholders})
                    ;"""
                )
                .format(
                    table=sql.Identifier(table_name),
                    placeholders=sql.SQL(", ").join(sql.Placeholder() * len(values[0])),
                )
            )
            cur.executemany(stmt, values)

except pg.OperationalError as e:
    logger.error(f"Connect to DB got error: {e}")

### table 'biowaste'

In [10]:
table_name = "biowaste"
values = [(r.meal_id, r.waste) for r in dish_biowaste.itertuples()]

try:
    with pg.connect(
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT,
        dbname=DB_NAME,
    ) as conn:
        with conn.cursor() as cur:
            stmt = (
                sql.SQL(
                    """
                    insert into {table} 
                    (meal_id, waste)
                    values ({placeholders})
                    ;"""
                )
                .format(
                    table=sql.Identifier(table_name),
                    placeholders=sql.SQL(", ").join(sql.Placeholder() * len(values[0])),
                )
            )
            cur.executemany(stmt, values)

except pg.OperationalError as e:
    logger.error(f"Connect to DB got error: {e}")

### table 'pieces_per_dish'

In [11]:
table_name = "pieces_per_dish"
values = [(r.meal_id, r.date, r.pcs) for r in pred_n_pcs_per_dish.itertuples()]

try:
    with pg.connect(
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT,
        dbname=DB_NAME,
    ) as conn:
        with conn.cursor() as cur:
            stmt = (
                sql.SQL(
                    """
                    insert into {table} 
                    (meal_id, date, pcs) 
                    values ({placeholders})
                    ;"""
                )
                .format(
                    table=sql.Identifier(table_name),
                    placeholders=sql.SQL(", ").join(sql.Placeholder() * len(values[0])),
                )
            )
            cur.executemany(stmt, values)

except pg.OperationalError as e:
    logger.error(f"Connect to DB got error: {e}")

### table 'pieces_whole'

In [12]:
table_name = "pieces_whole"
values = [(r.date, r.restaurant, r.pcs) for r in pred_n_pcs_whole.itertuples()]

try:
    with pg.connect(
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT,
        dbname=DB_NAME,
    ) as conn:
        with conn.cursor() as cur:
            stmt = (
                sql.SQL(
                    """
                    insert into {table} 
                    (date, restaurant, pcs) 
                    values ({placeholders})
                    ;"""
                )
                .format(
                    table=sql.Identifier(table_name),
                    placeholders=sql.SQL(", ").join(sql.Placeholder() * len(values[0])),
                )
            )
            cur.executemany(stmt, values)

except pg.OperationalError as e:
    logger.error(f"Connect to DB got error: {e}")

### table 'co2'

In [13]:
table_name = "co2"
values = [(r.meal_id, r.co2) for r in co2.itertuples()]

try:
    with pg.connect(
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT,
        dbname=DB_NAME,
    ) as conn:
        with conn.cursor() as cur:
            stmt = (
                sql.SQL(
                    """
                    insert into {table} 
                    (meal_id, co2) 
                    values ({placeholders})
                    ;"""
                )
                .format(
                    table=sql.Identifier(table_name),
                    placeholders=sql.SQL(", ").join(sql.Placeholder() * len(values[0])),
                )
            )
            cur.executemany(stmt, values)

except pg.OperationalError as e:
    logger.error(f"Connect to DB got error: {e}")

### table 'menus'

In [13]:
table_name = "menu"
values = [
    (
        r.date, 
        r.restaurant, 
        r.dish_1, 
        r.dish_2, 
        r.dish_3, 
        r.dish_4, 
        r.total_co2,
        r.total_waste,
        r.total_pcs_from_dishes,
        r.co2_per_customer,
        r.waste_per_customer,
        r.fitness
    ) 
    for r in menus.itertuples()
]

try:
    with pg.connect(
        user=USER,
        password=PWD,
        host=HOST,
        port=PORT,
        dbname=DB_NAME,
    ) as conn:
        with conn.cursor() as cur:
            stmt = (
                sql.SQL(
                    """
                    insert into {table} 
                    (
                        date
                        ,restaurant
                        ,dish_1
                        ,dish_2
                        ,dish_3
                        ,dish_4
                        ,total_co2
                        ,total_waste
                        ,total_pcs_from_dishes
                        ,co2_per_customer
                        ,waste_per_customer
                        ,fitness    
                    ) 
                    values ({placeholders})
                    ;"""
                )
                .format(
                    table=sql.Identifier(table_name),
                    placeholders=sql.SQL(", ").join(sql.Placeholder() * len(values[0])),
                )
            )
            cur.executemany(stmt, values)

except pg.OperationalError as e:
    logger.error(f"Connect to DB got error: {e}")